# Body Detection

Attempting to finetune Vgg16 model to solve https://www.kaggle.com/c/state-farm-distracted-driver-detection.

This notebook uses techniques described by the competition winner: https://www.kaggle.com/c/state-farm-distracted-driver-detection/discussion/22906.

In [ ]:
import importlib, sys
sys.path.insert(0, './../../utils')
import utils; importlib.reload(utils)
from utils import *

%matplotlib inline

K.set_image_dim_ordering('th')
base_dir = '../'
batch_size = 64
dropout = 0.7

start_w, start_h = 640, 480
end_w, end_h = 224, 224

## Utils

In [ ]:
def large_bb_to_small(bb):
    conv_x = (end_w / start_w)
    conv_y = (end_h / start_h)
    return [bb[0]*conv_x, bb[1]*conv_y, bb[2]*conv_x, bb[3]*conv_y]

def small_bb_to_large(bb):
    conv_x = (end_w / start_w)
    conv_y = (end_h / start_h)
    return [bb[0]/conv_x, bb[1]/conv_y, bb[2]/conv_x, bb[3]/conv_y]
    
def preprocess_img(img):
    resized_image = to_keras(scipy.misc.imresize(img, (end_w, end_h)))
    pred = dm.predict(cm.predict(np.array([resized_image])))[0]
    pred = np.array(small_bb_to_large(pred)).astype(np.int32)
    x = min(max(pred[0], 0), start_w-end_w)
    y = min(max(pred[1], 0), start_h-end_h)
    w = min(max(pred[2], end_w), start_w-x)
    h = min(max(pred[3], end_h), start_h-y)
    cropped_img = img[:, y:y+h, x:x+h]
    return to_keras(scipy.misc.imresize(cropped_img, (end_w, end_h)))

def gen_cropped_set(batches, dir_name):
    filenames = batches.filenames
    for f in tqdm(filenames):
        path = base_dir + dir_name + f
        img = Image.open(path)
        img.load()
        img = from_keras(preprocess_img(to_keras(np.asarray(img, dtype="int32"))))
        im = Image.fromarray(img)
        im.save(base_dir + 'cropped_' + dir_name + f)

## Create Batches

In [ ]:
gen = image.ImageDataGenerator()
trn_batches = gen.flow_from_directory(base_dir + 'bb_train/', target_size=(end_h,end_w), shuffle=False)
val_batches = gen.flow_from_directory(base_dir + 'bb_valid/', target_size=(end_h,end_w), shuffle=False)

trn_filenames = [f.split('/')[-1] for f in trn_batches.filenames]
val_filenames = [f.split('/')[-1] for f in val_batches.filenames]

## Manually Annotated Bounding Boxes

In [ ]:
trn_json = {}
with open(base_dir + 'trn_annotations.json') as file:
    trn_json = json.load(file)
    
val_json = {}
with open(base_dir + 'val_annotations.json') as file:
    val_json = json.load(file)
    
trn_bbox = np.stack([large_bb_to_small(trn_json[f]) for f in trn_filenames]).astype(np.float32)
val_bbox = np.stack([large_bb_to_small(val_json[f]) for f in val_filenames]).astype(np.float32)

In [ ]:
idx = 0

trn_batches.reset()
imgs = next(trn_batches)[0]
show_bb(imgs[idx], trn_bbox[idx])

val_batches.reset()
imgs = next(val_batches)[0]
show_bb(imgs[idx], val_bbox[idx])

## Convolutional Pre-Computing

In [ ]:
# Train/validation set pre-computing to speed up fine tuning
cm = conv_model(vgg(output_size, dropout=dropout))

trn_batches.reset()
trn_features = cm.predict_generator(trn_batches, steps(trn_batches, batch_size//2), verbose=1)
save_array(base_dir + 'models/bb_train_convlayer_features.bc', trn_features)

val_batches.reset()
val_features = cm.predict_generator(val_batches, steps(val_batches, batch_size//2), verbose=1)
save_array(base_dir + 'models/bb_valid_convlayer_features.bc', val_features)

In [ ]:
trn_features = load_array(base_dir + 'models/bb_train_convlayer_features.bc')
val_features = load_array(base_dir + 'models/bb_valid_convlayer_features.bc')

## Training Detection Model

In [ ]:
model = vgg(output_size, dropout=dropout)
model.load_weights(base_dir + 'models/aug_0.h5')
model.pop()
model.add(Dense(4))

dm = vgg_fc_model(model, 4, dropout=dropout, output_activation=None)
for i in range(len(dm.layers)): dm.layers[i].trainable = i >= len(dm.layers) - 3

fit_with_features(dm, RMSprop(1e-3), 60, trn_features, trn_bbox, val_features, val_bbox, loss='mse')
fit_with_features(dm, RMSprop(1e-4), 40, trn_features, trn_bbox, val_features, val_bbox, loss='mse')

for l in dm.layers: l.trainable = True
fit_with_features(dm, RMSprop(1e-3), 40, trn_features, trn_bbox, val_features, val_bbox, loss='mse')
fit_with_features(dm, RMSprop(1e-4), 30, trn_features, trn_bbox, val_features, val_bbox, loss='mse')
fit_with_features(dm, RMSprop(1e-5), 20, trn_features, trn_bbox, val_features, val_bbox, loss='mse')

for l1,l2 in zip(model.layers[last_conv_idx(model) + 1:], dm.layers): l1.set_weights(l2.get_weights())
model.save_weights(base_dir + 'models/bb0.h5')

## Generating Cropped Sets

In [ ]:
model = vgg(output_size, dropout=dropout)
model.pop()
model.add(Dense(4))
model.load_weights(base_dir + 'models/bb0.h5')
dm = vgg_fc_model(model, 4, dropout=dropout, output_activation=None)
cm = conv_model(vgg(output_size, dropout=dropout))

In [ ]:
idx = 0

# Show bounding box
val_batches.reset()
img = next(val_batches)[0][idx]
bb_pred = dm.predict(cm.predict(np.array([img])))[0]
show_bb_pred(img, val_bbox[idx], bb_pred)

# Show cropped image
filenames = val_batches.filenames
img = Image.open(base_dir + dir_name + '/' + filenames[idx])
img.load()
img = preprocess_img(to_keras(np.asarray(img, dtype="int32")))
plt.imshow(to_plot(img))

In [ ]:
for dir_name in ['valid/', 'train/', 'test/']:
    batches = gen.flow_from_directory(base_dir + dir_name, target_size=(end_h,end_w), class_mode='categorical', shuffle=False, batch_size=batch_size)
    gen_cropped_set(batches, dir_name)